In [1]:
import numpy as np

In [2]:
# number of elements
elements_no = int(input('enter the number of elements: '))

In [3]:
#define elements
elements = {}
for element in range(elements_no):
    elements[element] = {'L': float(input(f'enter the lenght of element (m) {element+1}')),
                         'E':float(input(f'enter the modulus of element {element+1}')),
                         'A':float(input(f'enter the area of element (m ^ 2) {element+1}')),
                         'cor':{'start':(float(input(f'enter the x coordinate of starting point of element {element+1}')), float(input(f'enter the y coordinate of starting point of element {element+1}'))),
                                'end':(float(input(f'enter the x coordinate of ending point of element {element+1}')), float(input(f'enter the y coordinate of ending point of element {element+1}'))),
                                },
                            'DOF':{0:{'index':int(input(f'enter the index of first degree of freedom for elemnt {element+1}')), 'C ':bool(int(input('is it free (0) or constrain (1):')))},
                                                 1:{'index':int(input(f'enter the index of second degree of freedom for elemnt {element+1}')), 'C ':bool(int(input('is it free (0) or constrain (1):')))},
                                                 2:{'index':int(input(f'enter the index of third degree of freedom for elemnt {element+1}')), 'C ':bool(int(input('is it free (0) or constrain (1):')))},
                                                 3:{'index':int(input(f'enter the index of fourth degree of freedom for elemnt {element+1}')), 'C ':bool(int(input('is it free (0) or constrain (1):')))}}

                         }

In [4]:
for element, feature in elements.items():
    feature['DOF'] =  dict(sorted(feature['DOF'].items(), key=lambda x: x[1]['index']))

In [5]:
DOFs = {}
for element, feature in elements.items():
    for i, dof in feature['DOF'].items():
        DOFs[dof['index']] = dof['C ']
DOFss = DOFs.keys()

In [6]:
FDOF = []
CDOF = []
for dof, c in DOFs.items():
    if c:
        CDOF.append(dof)
    else:
        FDOF.append(dof)
FDOF = sorted(FDOF)
CDOF = sorted(CDOF)

In [7]:
#define fixed forces matrix
Qf = np.zeros((len(DOFs), 1))

In [8]:
# define Qk
Qk = np.zeros((len(FDOF), 1))
for i, dof in enumerate(FDOF):
    Qk[i][0] = float(input(f'enter the external force exetred on degre of freedom No (N) {dof}'))

In [9]:
#define Qu
Qu = np.zeros((len(CDOF), 1))

In [10]:
#define Dk
Dk = np.zeros((len(CDOF), 1))
for i, dof in enumerate(CDOF):
    Dk[i][0] = float(input(f'enter the external displacemnet for degre of freedom No (m) {dof}'))

In [11]:
#define Qu
Du = np.zeros((len(FDOF), 1))

In [12]:
#local stifness matrix

matrix = [[1,-1],
               [-1,1]]

matrix = np.array(matrix)

stif_mat = {element:(elements[element]['E']*elements[element]['A']/elements[element]['L']) * matrix for element in elements}

In [13]:
stif_mat

{0: array([[ 50000., -50000.],
        [-50000.,  50000.]]),
 1: array([[ 40000., -40000.],
        [-40000.,  40000.]]),
 2: array([[ 66666.66666667, -66666.66666667],
        [-66666.66666667,  66666.66666667]]),
 3: array([[ 40000., -40000.],
        [-40000.,  40000.]]),
 4: array([[ 40000., -40000.],
        [-40000.,  40000.]])}

In [14]:
def lambda_x(xi, xj, yi, yj):
    return (xj - xi)/np.sqrt((xj-xi)**2 + (yj-yi)**2)

def lambda_y(xi, xj, yi, yj):
    return (yj - yi)/np.sqrt((xj-xi)**2 + (yj-yi)**2)

In [15]:
# transmition matrix
tr_mats = {element:np.array([[lambda_x(xi=elements[element]['cor']['start'][0], xj=elements[element]['cor']['end'][0],
                             yi=elements[element]['cor']['start'][1], yj=elements[element]['cor']['end'][1]),
                             lambda_y(xi=elements[element]['cor']['start'][0], xj=elements[element]['cor']['end'][0],
                             yi=elements[element]['cor']['start'][1], yj=elements[element]['cor']['end'][1]), 0, 0],
                             [0, 0,lambda_x(xi=elements[element]['cor']['start'][0], xj=elements[element]['cor']['end'][0],
                             yi=elements[element]['cor']['start'][1], yj=elements[element]['cor']['end'][1]),
                             lambda_y(xi=elements[element]['cor']['start'][0], xj=elements[element]['cor']['end'][0],
                             yi=elements[element]['cor']['start'][1], yj=elements[element]['cor']['end'][1]) ]]) for element in elements}

In [16]:
tr_mats

{0: array([[1., 0., 0., 0.],
        [0., 0., 1., 0.]]),
 1: array([[0.8, 0.6, 0. , 0. ],
        [0. , 0. , 0.8, 0.6]]),
 2: array([[0., 1., 0., 0.],
        [0., 0., 0., 1.]]),
 3: array([[ 0.8, -0.6,  0. ,  0. ],
        [ 0. ,  0. ,  0.8, -0.6]]),
 4: array([[ 0.8, -0.6,  0. ,  0. ],
        [ 0. ,  0. ,  0.8, -0.6]])}

In [18]:
elements_thermal_forces = {}
for element, feature in elements.items():
    is_thermal = bool(int(input(f'does element {element+1} have thermal force?')))
    if is_thermal:
        features =  {'delta T':float(input(f'enter the difference of thepruture for element {element}')), 'alpha':float(input(f'enter the thermal coefficent of element {element}'))}
        elements_thermal_forces[element] = {'thermal feature ':features, 'qf': np.dot(tr_mats[element].T, np.array([[features['alpha']*features['delta T']*feature['E']*feature['A']],
                                                                                                                [features['alpha']*features['delta T']*feature['E']*feature['A']*-1]])),
                                                                                                                }
    else:
        elements_thermal_forces[element] = { 'qf': np.dot(tr_mats[element].T, np.array([[0],
                                                                                        [0*-1]])),}

In [19]:
elements_error_forces = {}
for element, feature in elements.items():
    features =  {'error':float(input(f'enter the lenght error for element {element+1}'))}
    elements_error_forces[element] = {'error feature ':features, 'qf': np.dot(tr_mats[element].T, np.array([[features['error']*feature['E']*feature['A']],
                                                                                                               [features['error']*feature['E']*feature['A']*-1]])),
                                                                                                               }

In [20]:
#total QFk
Qfk = np.zeros((len(FDOF), 1))
for i, dof in enumerate(FDOF):
    
    for element, feature in elements.items():
        dofs = []
        for _, dof_2 in feature['DOF'].items():
            dofs.append(dof_2['index'])
        for j, eldof in enumerate(dofs):
            if dof == eldof:
                
                Qfk[i][0] += elements_thermal_forces[element]['qf'][j][0]
                Qfk[i][0] += elements_error_forces[element]['qf'][j][0]


In [21]:
#total QFu
Qfu = np.zeros((len(CDOF), 1))
for i, dof in enumerate(CDOF):
    
    for element, feature in elements.items():
        dofs = []
        for _, dof_2 in feature['DOF'].items():
            dofs.append(dof_2['index'])
        for j, eldof in enumerate(dofs):
            if dof == eldof:
                
                Qfu[i][0] += elements_thermal_forces[element]['qf'][j][0]
                Qfu[i][0] += elements_error_forces[element]['qf'][j][0]

In [22]:
Qfu

array([[384.],
       [288.],
       [  0.],
       [  0.]])

In [23]:
#global stiffnes matrix
k = {}
for element, feature in elements.items():
    xi = feature['cor']['start'][0]
    xj = feature['cor']['end'][0]
    yi = feature['cor']['start'][1]
    yj = feature['cor']['end'][1]
    k[element] = (feature['E'] * feature['A'] / feature['L']) * np.array([[lambda_x(xi, xj, yi, yj)*lambda_x(xi, xj, yi, yj), lambda_x(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj),
                                                                            -1*lambda_x(xi, xj, yi, yj)*lambda_x(xi, xj, yi, yj), -1*lambda_x(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj)],

                                                                            [lambda_x(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj), lambda_y(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj),
                                                                             -1*lambda_x(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj), -1*lambda_y(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj)],

                                                                             [-1*lambda_x(xi, xj, yi, yj)*lambda_x(xi, xj, yi, yj), -1*lambda_x(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj),
                                                                              lambda_x(xi, xj, yi, yj)*lambda_x(xi, xj, yi, yj),lambda_x(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj) ],

                                                                              [-1*lambda_x(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj),-1*lambda_y(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj),
                                                                               lambda_x(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj), lambda_y(xi, xj, yi, yj)*lambda_y(xi, xj, yi, yj)]
                                                                              ])

In [24]:
kps = {}
for element, feature in elements.items():    
    dofs = []
    for _, dof in feature['DOF'].items():
        dofs.append(dof['index'])
    kp = np.empty((4,4))
    for i, v in enumerate(dofs):
        for j, m in enumerate(dofs):
            kp[i][j] = str(v) + str(m)
    kps[element] = kp


In [25]:
k

{0: array([[ 50000.,      0., -50000.,     -0.],
        [     0.,      0.,     -0.,     -0.],
        [-50000.,     -0.,  50000.,      0.],
        [    -0.,     -0.,      0.,      0.]]),
 1: array([[ 25600.,  19200., -25600., -19200.],
        [ 19200.,  14400., -19200., -14400.],
        [-25600., -19200.,  25600.,  19200.],
        [-19200., -14400.,  19200.,  14400.]]),
 2: array([[     0.        ,      0.        ,     -0.        ,
             -0.        ],
        [     0.        ,  66666.66666667,     -0.        ,
         -66666.66666667],
        [    -0.        ,     -0.        ,      0.        ,
              0.        ],
        [    -0.        , -66666.66666667,      0.        ,
          66666.66666667]]),
 3: array([[ 25600., -19200., -25600.,  19200.],
        [-19200.,  14400.,  19200., -14400.],
        [-25600.,  19200.,  25600., -19200.],
        [ 19200., -14400., -19200.,  14400.]]),
 4: array([[ 25600., -19200., -25600.,  19200.],
        [-19200.,  14400.,  192

In [26]:
# K ff
Kff = np.zeros((len(FDOF), len(FDOF)))
for i, v in enumerate(FDOF):
    for j, m in enumerate(FDOF):
        for element, feature in elements.items():
            dofs = []
            for _, dof in feature['DOF'].items():
                dofs.append(dof['index'])
            for I, V in enumerate(dofs):
                for J, K in enumerate(dofs):
                    if float(str(v) + str(m)) == kps[element][I][J]:
                        Kff[i][j] += k[element][I][J]

In [27]:
Kff

array([[ 81066.66666667,  19200.        , -14400.        ,
             0.        ],
       [ 19200.        ,  76800.        , -19200.        ,
        -25600.        ],
       [-14400.        , -19200.        ,  43200.        ,
         19200.        ],
       [     0.        , -25600.        ,  19200.        ,
         75600.        ]])

In [28]:
# K fc
Kfc = np.zeros((len(FDOF), len(CDOF)))
for i, v in enumerate(FDOF):
    for j, m in enumerate(CDOF):
        for element, feature in elements.items():
            dofs = []
            for _, dof in feature['DOF'].items():
                dofs.append(dof['index'])
            for I, V in enumerate(dofs):
                for J, K in enumerate(dofs):
                    if float(str(v) + str(m)) == kps[element][I][J]:
                        Kfc[i][j] += k[element][I][J]

In [29]:
Kfc

array([[     0.        , -66666.66666667, -19200.        ,
             0.        ],
       [-25600.        , -19200.        , -25600.        ,
         19200.        ],
       [-19200.        , -14400.        ,  19200.        ,
        -14400.        ],
       [-50000.        ,      0.        ,      0.        ,
        -19200.        ]])

In [30]:
# K cf
Kcf = np.zeros((len(CDOF), len(FDOF)))
for i, v in enumerate(CDOF):
    for j, m in enumerate(FDOF):
        for element, feature in elements.items():
            dofs = []
            for _, dof in feature['DOF'].items():
                dofs.append(dof['index'])
            for I, V in enumerate(dofs):
                for J, K in enumerate(dofs):
                    if float(str(v) + str(m)) == kps[element][I][J]:
                        Kcf[i][j] += k[element][I][J]

In [31]:
Kcf

array([[     0.        , -25600.        , -19200.        ,
        -50000.        ],
       [-66666.66666667, -19200.        , -14400.        ,
             0.        ],
       [-19200.        , -25600.        ,  19200.        ,
             0.        ],
       [     0.        ,  19200.        , -14400.        ,
        -19200.        ]])

In [32]:
# K cc
Kcc = np.zeros((len(CDOF), len(CDOF)))
for i, v in enumerate(CDOF):
    for j, m in enumerate(CDOF):
        for element, feature in elements.items():
            dofs = []
            for _, dof in feature['DOF'].items():
                dofs.append(dof['index'])
            for I, V in enumerate(dofs):
                for J, K in enumerate(dofs):
                    if float(str(v) + str(m)) == kps[element][I][J]:
                        Kcc[i][j] += k[element][I][J]

In [33]:
Kcc

array([[75600.        , 19200.        ,     0.        ,     0.        ],
       [19200.        , 81066.66666667,     0.        ,     0.        ],
       [    0.        ,     0.        , 25600.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , 14400.        ]])

In [35]:
Du = np.dot(np.linalg.inv(Kff),(Qk - Qfk - np.dot(Kfc, Dk)))

In [36]:
Qu = np.dot(Kff, Du)+np.dot(Kfc, Dk)+Qfu

In [37]:
Du

array([[-0.52053226],
       [ 1.42981116],
       [-1.01935932],
       [ 0.74305376]])

In [38]:
Qu

array([[ 3.84000000e+02],
       [ 1.00672000e+05],
       [-4.97120000e+04],
       [-3.63797881e-12]])